In [1]:
%load_ext autoreload
%autoreload 2

!! pip install geopandas 
import sys
sys.path.append("..")

import pandas as pd
import numpy as np
import geopandas as gpd
from matplotlib import pyplot as plt
from pathlib import Path
from collections import Counter, defaultdict
from tqdm.auto import tqdm
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, cross_validate
import src.data_utils as u
import src.model_utils as m
from scipy.stats import kendalltau


# avg events hour rate: 0.37% 
DATA = Path('../data/')
RESULT = Path('../results/')

train_path = DATA / 'train.csv'
shape_path = DATA / 'road_segments' / 'road_segments.shp'
velocity_path = DATA / 'data_velocity.csv'

# Injurence and velocities

In [24]:
train = pd.read_csv(train_path, parse_dates = ['Occurrence Local Date Time'])


inj = pd.read_csv(DATA / 'SANRAL_v2' / 'Injuries2016_2019.csv', parse_dates=['Created Local Date Time'])
inj = inj.rename(columns={'Created Local Date Time': 'time'})
inj = u.select_by_time(inj, '2016-01-01', '2019-01-01')


veh = pd.read_csv(DATA / 'SANRAL_v2' / 'Vehicles2016_2019.csv')
veh = veh.fillna(0)
veh['EventID'] = veh.EventID.astype(int)
veh = veh.head(n=63469)

In [26]:
print(
len(set(train.EventId) - (set(veh.EventID) | set(inj['Event Id']))),
len((set(veh.EventID) | set(inj['Event Id'])) - set(train.EventId)),
len(set(inj['Event Id']).intersection(set(veh.EventID)))
)

9218 12565 13909


# Correlations

In [ ]:
data_ones = u.read_ones(train_path)

data_a = u.select_by_time(data_ones, '2016-01-01', '2016-12-31')
data_b = u.select_by_time(data_ones, '2018-01-01', '2018-12-31')
data_c = u.select_by_time(data_ones, '2017-01-01', '2017-12-31')

union_sids = set(data_a.sid).intersection(set(data_b.sid)).intersection(set(data_c.sid))

data_a = data_a[data_a.sid.isin(union_sids)]
data_b = data_b[data_b.sid.isin(union_sids)]
data_c = data_c[data_c.sid.isin(union_sids)]

count_a = data_a.groupby('sid').count().sort_values(by='sid')
count_b = data_b.groupby('sid').count().sort_values(by='sid')
count_c = data_c.groupby('sid').count().sort_values(by='sid')


print(kendalltau(count_a.target, count_b.target))
print(kendalltau(count_a.target, count_c.target))
print(kendalltau(count_b.target, count_c.target))
